In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/Foodie_CLIP

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/Foodie_CLIP


In [ ]:
!pip install transformers
!pip install torcheval
!pip install torch

In [1]:
from foodie_clip.foodieclip import FoodieClip, ClIPLoss
from trainer.utils import CLIPDataCollator, CLIPDataset
from trainer.evaluate import compute_accuracy
from foodie_clip.foodieclip import FoodieClip, ClIPLoss
from trainer.utils import CLIPDataset, CLIPDataCollator
from trainer.clip_trainer import CLIPTrainer
from transformers import (
    TrainingArguments,
    AutoProcessor,
    CLIPConfig,
    EarlyStoppingCallback,
)

/Users/yusali/dev/Foodie_CLIP/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import json

# Read the config.json file
with open('config.json') as f:
    config = json.load(f)

# Access the config parameters
DATA_DIRECTORY = config["DATA_DIRECTORY"]

print("DATA_DIRECTORY: ", DATA_DIRECTORY)

DATA_DIRECTORY:  data/


In [3]:
import torch

# Check if MPS is available and set the device
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print("Training on device: ", device)

Training on device:  mps


In [5]:
# loading model
foodieclip_config = CLIPConfig.from_pretrained("openai/clip-vit-base-patch32")
foodieclip = FoodieClip(foodieclip_config, projection_dim=128).to(device)
# loading dataset
train_dataset = CLIPDataset(train=True,path=DATA_DIRECTORY, class_max_datapoints=200)
print(len(train_dataset))
eval_dataset = CLIPDataset(train=False, path=DATA_DIRECTORY, class_max_datapoints=100)
# set data collator
processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")
data_collator = CLIPDataCollator(processor=processor)
# set up training args
training_args = TrainingArguments(
    output_dir="output/",
    per_device_train_batch_size=500,
    evaluation_strategy="steps",
    logging_steps=1,
    num_train_epochs=2,
    report_to=None,
    eval_steps=5,
    fp16=False,
    remove_unused_columns=False,
    load_best_model_at_end=True,
)
trainer = CLIPTrainer(
    model=foodieclip,
    training_args=training_args,
    data_collator=data_collator,
    max_length=128,
    compute_metrics=compute_accuracy,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    Loss_FN=ClIPLoss,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)
trainer.train()

47177


/Users/yusali/dev/Foodie_CLIP/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/yusali/dev/Foodie_CLIP/venv/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 0/190 [00:00<?, ?it/s]

RuntimeError: MPS backend out of memory (MPS allocated: 18.03 GB, other allocations: 12.69 MB, max allowed: 18.13 GB). Tried to allocate 57.22 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).